In [104]:
import pandas as pd

In [105]:
df_de = pd.read_csv('../grams/sg_de.csv', names=['frequency', 'word1', 'word2']) # results of skip-grams and 2-grams
df_nl = pd.read_csv('../grams/sg_nl.csv', names=['frequency', 'word1', 'word2']) # results of skip-grams and 2-grams

In [106]:
# list of ~94k US babynames to remove (hello, john), (james, bond)
names = pd.read_csv("../names.csv", names=['name'])['name'].tolist()

In [107]:
names = [i for i in names if type(i) == str] #remove non strings

# Filter names and from list

In [108]:
german_filter = ['ich', 'du', 'sie', 'mir', 'wir', 'ihr', 'uns', 'euch', 'die', 'das', 'der', 'den', 'er', 'und', '!', '/', 'ein', 'eine', 'einer', 'einem', 'mein', 'dein', 'nicht', 'wie', 'wo', 'wann', 'was']

In [109]:
dutch_filter = ['ik', 'jij', 'hij', 'zij', 'wij', 'we', 'jullie', 'hun', 'mij', 'me', 'mijn', 'haar', 'hem', '!', '/', 'een', 'de', 'het', 'wie', 'wat', 'waar', 'hoe', 'niet']

In [110]:
l_de = names + german_filter
l_nl = names + dutch_filter

In [111]:
df_de = \
    df_de[ \
    ~df_de.word1.isin(l_de) & \
    ~df_de.word2.isin(l_de) & \
    ~df_de.word1.str.isnumeric() & \
    ~df_de.word2.str.isnumeric() &\
    ~(df_de.word1.str == df_de.word2.str)
    ]
    
df_nl = \
    df_nl[ \
    ~df_nl.word1.isin(l_nl) & \
    ~df_nl.word2.isin(l_nl) & \
    ~df_nl.word1.str.isnumeric() & \
    ~df_nl.word2.str.isnumeric() & \
    ~(df_de.word1.str == df_de.word2.str)
    ]

# Remove words overlapping with other languages

In [138]:
df_nl_ex = df_nl \
    .merge(df_de ,left_on=['word1', 'word2'], right_on=['word1', 'word2'], how='left')

In [139]:
df_nl_ex.frequency_y.fillna(0, inplace=True)

In [140]:
df = df_nl_ex[df_nl_ex.frequency_y == 0.0].drop('frequency_y', axis=1).rename(index=str, columns={"frequency_x": "frequency"})

# Add word frequency

In [141]:
wc_de = pd.read_csv("../word_counts/wc_de.csv", lineterminator='\n', names=['word', 'word_frequency'])

In [142]:
df = df.merge(wc_de, left_on='word1', right_on='word', how='left') \
.merge(wc_de, left_on='word2', right_on='word', how='left') \
.drop(['word_x', 'word_y'], axis=1) \
.rename(index=str, columns={"word_frequency_x": "word_1_frequency", "word_frequency_y": "word_2_frequency"})

In [143]:
df['normalised_frequency'] = df['frequency'] / (df['word_1_frequency']+df['word_2_frequency']) # divide skipgram freq. by wordcount freq.

In [144]:
df = df.sort_values(by='frequency', ascending=False)

In [146]:
df[df['word2'] == "aan"]

,frequency,word1,word2,word_1_frequency,word_2_frequency,normalised_frequency
369663,365152,er,aan,1907454.0,8.0,0.191433
624230,309547,is,aan,9836.0,8.0,31.445246
238451,144200,maar,aan,11.0,8.0,7589.473684
530578,117279,als,aan,611492.0,8.0,0.191789
78868,111984,die,aan,3505417.0,8.0,0.031946
513858,97507,zijn,aan,NaN,8.0,NaN
379877,94328,heb,aan,2971.0,8.0,31.664317
653940,82748,was,aan,2387393.0,8.0,0.034660
119910,73422,denk,aan,21243.0,8.0,3.454990
85748,72363,met,aan,565.0,8.0,126.287958
